In [1]:
""" 1. Export your OpenApi Spec file, then make an instance of the OpenApi runner. """
from pyclinic.openapi import OpenApi
from rich import print

collection_path = "./tests/examples/petstore.openapi3.yaml"
runner = OpenApi(collection_path)

Variable {petId} was not found in the Variables provided
Variable {petId} was not found in the Variables provided
Variable {petId} was not found in the Variables provided
Variable {petId} was not found in the Variables provided
Variable {orderId} was not found in the Variables provided
Variable {orderId} was not found in the Variables provided
Variable {username} was not found in the Variables provided
Variable {username} was not found in the Variables provided
Variable {username} was not found in the Variables provided


In [2]:
""" 2. Did you see the warnings that were printed above?

That's because there are variables, like {petId}, that exist in the Spec file but don't have values!
So calling the functions as-is will fail.

However, you can solve this by passing in User Variables when instantiating the OpenApi runner
or when you call the functions. (We will cover this later in the notebook)

But, for now, let's see which variables we _do_ have defined. 
"""
runner.show_variables()

──────────────────────────── User Variables for Swagger Petstore ────────────────────────────

{}

In [3]:
""" 3. An empty dictionary means that there are zero variables defined...

Let's see what folders and functions we have.
"""
runner.show_folders()

──────────────────────────────── Folders in Swagger Petstore ────────────────────────────────

Pet.

        add_pet

        delete_pet

        find_pets_by_status

...

        login_user

        logout_user

        update_user

Example Usage

response = runner.User.delete_user()

In [5]:
""" 4. We have a lot and they're ready to be used!

A few things to note:
    - Each class name (aka folder like "Pet") shows which functions belong to it
        * This makes it easy to execute doing something like runner.User.delete_user()
        * Just like it shows in the Example Usage
    
    - You'll get errors if you try to use a function that doesn't exist in a folder
    
    * We're about to see an error because we're missing the {petId} variable...
"""
# runner.Pet.add_a_new_pet() # This will fail because add_a_new_pet is not defined in the Pet folder

response = runner.Pet.get_pet_by_id()
print("STATUS CODE:", response.status_code)
print(response.text)

STATUS CODE: 404

{"code":404,"type":"unknown","message":"java.lang.NumberFormatException: For input string: 
\"{petId}\""}

In [7]:
""" 5. Notice how we're working with a response. This comes from the requests library.

Using type hinting, we will get more intellisense. Now we can work with the response a lot more.

Let's use an auto-generated function that we know will work without needing variables.
"""
from requests import Response

payload = {
        "id": 0,
        "category": {"id": 0, "name": "string"},
        "name": "doggie",
        "photoUrls": ["string"],
        "tags": [{"id": 0, "name": "string"}],
        "status": "available",
    }
response = runner.Pet.add_pet(json=payload)
pet_id = response.json()["id"]
print(response.json())

{
    'id': 9223372036854775807,
    'category': {'id': 0, 'name': 'string'},
    'name': 'doggie',
    'photoUrls': ['string'],
    'tags': [{'id': 0, 'name': 'string'}],
    'status': 'available'
}

In [8]:
""" 6. AMAZING! Let's try finding a pet again, but this time we'll pass in a valid {petId}. """
response = runner.Pet.get_pet_by_id({"petId": pet_id})
print("STATUS CODE:", response.status_code)
print(response.json())

STATUS CODE: 200

{
    'id': 9223372036854775807,
    'name': 'siyaan',
    'photoUrls': [],
    'tags': [],
    'status': 'available'
}

In [9]:
""" 7. SUCCESS! If you wanted this to be an Automated Test with pytest, it would look something like this: """
def test_create_pet():
    payload = {
        "id": 0,
        "category": {"id": 0, "name": "string"},
        "name": "doggie",
        "photoUrls": ["string"],
        "tags": [{"id": 0, "name": "string"}],
        "status": "available",
    }
    runner = OpenApi(collection_path)
    response = runner.Pet.add_pet(json=payload)

    assert response.ok
    assert response.json()["id"] > 0
    assert response.json()["status"] == "available"

In [10]:
""" 8. User Variables

There are two places where you can pass in User Variables:

1. When instantiating the OpenApi runner. This will set the variables for all functions in the runner.
2. When calling the functions. This only sets the variables for that function.

* Variables are a flat dictionary of key-value pairs that _are_ case-sensitive.
"""

user_variables = {"USERNAME": "Carlos Kidman"}
runner = OpenApi(collection_path, user_variables)
runner.show_variables()

Variable {petId} was not found in the Variables provided
Variable {petId} was not found in the Variables provided
Variable {petId} was not found in the Variables provided
Variable {petId} was not found in the Variables provided
Variable {orderId} was not found in the Variables provided
Variable {orderId} was not found in the Variables provided
Variable {username} was not found in the Variables provided
Variable {username} was not found in the Variables provided
Variable {username} was not found in the Variables provided


──────────────────────────── User Variables for Swagger Petstore ────────────────────────────

{'USERNAME': 'Carlos Kidman'}

In [12]:
""" 9. Variables at the function level """

# Each function has a .help() method to display what the request looks like before being sent
runner.Pet.get_pet_by_id.help()

# It shows that {petId} is a variable that we need to pass in
runner.Pet.get_pet_by_id({"petId": 123})

# No pet with the ID of 123 exists, so we get a 404, but that's expected!

───────────────────────────── Request to Send for get_pet_by_id ─────────────────────────────

{'method': 'GET', 'url': 'https://petstore.swagger.io/v2/pet/{petId}'}

<Response [404]>

In [18]:
""" 10. Create your own Service functions to extend or add functionality and power! """

def get_sold_count_for_today():
    response = runner.Store.get_inventory()
    inventory = response.json()
    return inventory.get("sold", 0)


sold = get_sold_count_for_today()
print("PETS SOLD TODAY:", sold)

PETS SOLD TODAY: 60